In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import re
import warnings
warnings.filterwarnings('ignore')
import cmc_parser as cp


# Individual file parsing

In [2]:
def parse_bhformation_dat(model_path):

    filepath= model_path+'/initial.bhformation.dat'
    # Define column names for the core data we want to keep
    column_names = [
        'time', 'r', 'binary', 'ID', 'zams_m', 'm_progenitor', 'bh_mass',
        'bh_spin', 'birth_kick'
    ]

    # Read the file, skipping any lines that start with #
    data = []
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('#'):
                # Split the line by whitespace
                values = line.strip().split()
                if len(values) >= 9:  # Ensure we have enough values for our columns
                    # Only take the first 9 values (excluding vs columns)
                    data.append(values[:9])

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=column_names)

    # Convert columns to appropriate data types
    numeric_cols = ['time', 'r', 'zams_m', 'm_progenitor', 'bh_mass', 'bh_spin']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])

    # Convert binary and ID to integers
    df['binary'] = df['binary'].astype(int)
    df['ID'] = df['ID'].astype(int)

    # Handle birth_kick column
    df['birth_kick'] = pd.to_numeric(df['birth_kick'], errors='coerce')

    return df
parse_bhformation_dat('data/N2.0rv0.5rg2.0Z0.0002')

,time,r,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick
0,0.009197,0.190420,1,124149,0.000000,47.24320,42.51890,0,0.0000
1,0.009476,0.784866,0,178115,104.318000,101.11100,40.50000,0,1.0000
2,0.009920,0.069105,1,150155,0.000000,37.76810,33.99130,0,0.0000
3,0.010120,0.109359,1,228329,0.000000,35.36190,31.82570,0,0.0000
4,0.010364,1.755900,0,9114,0.181263,32.65030,29.38530,0,1.0000
...,...,...,...,...,...,...,...,...,...
338,0.039135,0.127507,1,216993,0.000000,75.53730,67.98360,0,0.0000
339,0.040909,0.233458,0,185677,0.178679,25.05900,9.83723,0,151.7321
340,0.043852,0.321977,0,238752,0.336246,21.79050,19.61150,0,1.0000
341,6.507828,0.035967,1,0,0.000000,4.23882,16.73940,0,0.0000


In [3]:
def parse_collision_log(model_path):


    filepath = model_path + '/initial.collision.log'
    data = {
        'time': [],
        'collision_type': [],
        'idm': [],
        'mm': [],
        'id1': [],
        'm1': [],
        'id2': [],
        'm2': [],
        'r': [],
        'typem': [],
        'type1': [],
        'type2': []
    }

    def add_collision_record(time, collision_type, idm, mm,
                             id1, m1, id2, m2,
                             r, typem, type1, type2):
        data['time'].append(time)
        data['collision_type'].append(collision_type)
        data['idm'].append(idm)
        data['mm'].append(mm)
        data['id1'].append(id1)
        data['m1'].append(m1)
        data['id2'].append(id2)
        data['m2'].append(m2)
        data['r'].append(r)
        data['typem'].append(typem)
        data['type1'].append(type1)
        data['type2'].append(type2)

    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('t='):
                continue

            # common to all
            time_match   = re.search(r't=([\d.eE+-]+)', line)
            type_match   = re.search(r'\s(binary-[a-z]+|single-single)\s', line)
            idm_mm_match = re.search(r'idm=(\d+)\(mm=([\d.eE+-]+)\)', line)
            r_match      = re.search(r'\(r=([\d.eE+-]+)\)', line)
            typem_match  = re.search(r'typem=(\d+)', line)

            # skip weird lines 
            if not all([time_match, type_match, idm_mm_match,
                        r_match, typem_match]):
                continue

            time       = float(time_match.group(1))
            collision_type = type_match.group(1)
            original_idm   = int(idm_mm_match.group(1))
            original_mm    = float(idm_mm_match.group(2))
            r              = float(r_match.group(1))
            typem          = int(typem_match.group(1))

            idm_block   = re.findall(r'id(\d)=(\d+)\(m\1=([\d.eE+-]+)\)', line)
            type_block  = re.findall(r'type(\d)=(\d+)', line)

            bodies = [(int(idx), int(star_id), float(mass))
                      for idx, star_id, mass in idm_block]
            type_dict = {int(idx): int(t) for idx, t in type_block}

            # need at least a binary encounter
            if len(bodies) < 2:
                continue

            # sort by mass (descending) while preserving idx for type lookup
            bodies_sorted = sorted(
                [(star_id, mass, type_dict[idx])
                 for idx, star_id, mass in bodies],
                key=lambda x: x[1], reverse=True)

            # ----------------------------------------------------------------
            # 2-body collision  → write one row unchanged
            # ----------------------------------------------------------------
            if len(bodies_sorted) == 2:
                (id1, m1, t1), (id2, m2, t2) = bodies_sorted
                add_collision_record(
                    time=time,
                    collision_type=collision_type,
                    idm=original_idm,
                    mm=original_mm,
                    id1=id1, m1=m1,
                    id2=id2, m2=m2,
                    r=r,
                    typem=typem,
                    type1=t1, type2=t2
                )
                continue

            # ----------------------------------------------------------------
            # 3-body collision  → two rows (big+mid, product+small)
            # ----------------------------------------------------------------
            if len(bodies_sorted) == 3:
                (big_id, big_m, big_t), \
                (mid_id, mid_m, mid_t), \
                (sml_id, sml_m, sml_t) = bodies_sorted

                # Collision 1
                cid1 = big_id + mid_id
                cm1  = big_m  + mid_m
                add_collision_record(time, collision_type,
                                     cid1, cm1,
                                     big_id, big_m,
                                     mid_id, mid_m,
                                     r, typem,
                                     big_t, mid_t)

                # Collision 2  (final product keeps original idm/mm)
                add_collision_record(time, collision_type,
                                     original_idm, original_mm,
                                     cid1, cm1,
                                     sml_id, sml_m,
                                     r, typem,
                                     big_t,       sml_t) # NOTE I CHANGED THE -1 HERE TO BIG_T
                continue

            # ----------------------------------------------------------------
            # 4-body collision (two binaries)  → three rows
            # scheme: (big1+big2) → (prod+small1) → (prod+small2)
            # ----------------------------------------------------------------
            if len(bodies_sorted) == 4:
                (b1_id, b1_m, b1_t), \
                (b2_id, b2_m, b2_t), \
                (s1_id, s1_m, s1_t), \
                (s2_id, s2_m, s2_t) = bodies_sorted

                # Collision 1: two big stars
                cid1 = b1_id + b2_id
                cm1  = b1_m  + b2_m
                add_collision_record(time, collision_type,
                                     cid1, cm1,
                                     b1_id, b1_m,
                                     b2_id, b2_m,
                                     r, typem,
                                     b1_t, b2_t)

                # Collision 2: product + first small star
                cid2 = cid1 + s1_id
                cm2  = cm1  + s1_m
                add_collision_record(time, collision_type,
                                     cid2, cm2,
                                     cid1, cm1,
                                     s1_id, s1_m,
                                     r, typem,
                                     -1,     s1_t)

                # Collision 3: product + second small star (final)
                add_collision_record(time, collision_type,
                                     original_idm, original_mm,
                                     cid2, cm2,
                                     s2_id, s2_m,
                                     r, typem,
                                     -1,     s2_t)
                continue


            raise ValueError(
                f"Encounter with {len(bodies_sorted)} participants "
            )

    df = pd.DataFrame(data)
    return df


In [4]:
def parse_merger_file(model_path):
    filepath = model_path + '/initial.semergedisrupt.log'
    # Initialize lists to store the extracted data
    data = {
        'time': [],
        'interaction_type': [],
        'id_rem': [],
        'mass_rem': [],
        'id1': [],
        'm1': [],
        'id2': [],
        'm2': [],
        'r': [],
        'type_rem': [],
        'type1': [],
        'type2': []
    }

    with open(filepath, 'r') as file:
        for line in file:
            # Skip comment lines
            if line.startswith('#'):
                continue

            # Check if this is a data line
            if line.startswith('t='):
                # Extract time
                time_match = re.search(r't=([0-9.eE+-]+)', line)
                time = float(time_match.group(1)) if time_match else np.nan

                # Extract interaction type
                type_match = re.search(r't=[\d.eE+-]+\s+(\w+)', line)
                interaction_type = type_match.group(1) if type_match else np.nan

                # Extract remnant ID and mass
                id_rem_match = re.search(r'idr=(\d+)\(mr=([\d.eE+-]+)\)', line)
                id_rem = int(id_rem_match.group(1)) if id_rem_match else np.nan
                mass_rem = float(id_rem_match.group(2)) if id_rem_match else np.nan

                # Extract id1 and m1
                id1_m1_match = re.search(r'id1=(\d+)\(m1=([\d.eE+-]+)\)', line)
                id1 = int(id1_m1_match.group(1)) if id1_m1_match else np.nan
                m1 = float(id1_m1_match.group(2)) if id1_m1_match else np.nan

                # Extract id2 and m2
                id2_m2_match = re.search(r'id2=(\d+)\(m2=([\d.eE+-]+)\)', line)
                id2 = int(id2_m2_match.group(1)) if id2_m2_match else np.nan
                m2 = float(id2_m2_match.group(2)) if id2_m2_match else np.nan

                # Extract radius
                r_match = re.search(r'\(r=([\d.eE+-]+)\)', line)
                r = float(r_match.group(1)) if r_match else np.nan

                # Extract types
                type_rem_match = re.search(r'typer=(\d+)', line)
                type1_match = re.search(r'type1=(\d+)', line)
                type2_match = re.search(r'type2=(\d+)', line)

                type_rem = int(type_rem_match.group(1)) if type_rem_match else np.nan
                type1 = int(type1_match.group(1)) if type1_match else np.nan
                type2 = int(type2_match.group(1)) if type2_match else np.nan

                # Append data
                data['time'].append(time)
                data['interaction_type'].append(interaction_type)
                data['id_rem'].append(id_rem)
                data['mass_rem'].append(mass_rem)
                data['id1'].append(id1)
                data['m1'].append(m1)
                data['id2'].append(id2)
                data['m2'].append(m2)
                data['r'].append(r)
                data['type_rem'].append(type_rem)
                data['type1'].append(type1)
                data['type2'].append(type2)

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
merger_df = parse_merger_file('data/N2.0rv0.5rg2.0Z0.0002')

merger_df


,time,interaction_type,id_rem,mass_rem,id1,m1,id2,m2,r,type_rem,type1,type2
0,0.001989,disrupt1,312311.0,75.21870,312311,74.964600,162670,0.254204,0.314027,1.0,1,0
1,0.005335,disrupt1,136651.0,89.76560,136651,89.665800,125486,0.099913,0.221251,1.0,1,0
2,0.005919,disrupt1,161709.0,83.86030,161709,78.209600,204653,5.650770,0.063689,1.0,1,1
3,0.007645,disrupt1,325918.0,78.02040,325918,77.910400,158314,0.110015,0.062531,1.0,1,0
4,0.008486,disrupt2,175727.0,109.86200,161895,0.092841,175727,109.886000,0.181820,2.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
618,9.037510,disruptboth,NaN,NaN,210009,1.664140,160681,1.102050,0.012559,NaN,8,1
619,9.157400,disrupt2,287355.0,2.35583,155861,0.980142,287355,1.870000,0.017850,5.0,11,6
620,9.162860,disrupt2,378095.0,2.80844,162500,0.876363,378095,2.070240,0.008220,4.0,11,5
621,9.181540,disruptboth,NaN,NaN,219323,1.107870,125797,1.202410,0.011862,NaN,5,11


# parsing bh mergers across models

In [5]:
def parse_merger_files(*filenames):
    """Return a single DataFrame with all rows from the supplied catalogue files."""
    i=0
    # setting up col properties
    PROPERTIES = [
    ("model_num",      float),    # 0
    ("rv",             float),  # 1
    ("rg",             float),    # 2
    ("Z",              float),  # 3
    ("N",              float),    # 4  (×10^5)
    ("merger_time",    float),  # 5  (Myr)
    ("id1",            float),    # 6
    ("id2",            float),    # 7
    ("m1",             float),  # 8  (M⊙)
    ("m2",             float),  # 9
    ("spin1",          float),  # 10
    ("spin2",          float),  # 11
    ("v_kick",         float),  # 12 (km s⁻¹)
    ("v_esc",          float),  # 13
    ("merger_channel", float),    # 14
    ("id3",            float),    # 15 (remnant id)
    ("m3",             float),  # 16
    ("spin3",          float),  # 17
    ("a_final",        float),  # 18 (AU)
    ("e_final",        float),  # 19
    ("e_10hz",         float)   # 20
    ]
    COLS = [c for c, _ in PROPERTIES]
    records = []
    for fname in filenames:
        try:
            with open(fname) as fh:
                for line in fh:
                    if line.startswith("#") or not line.strip():
                        continue
                    parts = line.split()
                    if len(parts) < len(PROPERTIES):
                        print(line)
                        # silently skip malformed lines
                        continue
                    rec = {col: typ(parts[i]) for i, (col, typ) in enumerate(PROPERTIES)}
                    records.append(rec)
        except FileNotFoundError:
            print(f"  File '{fname}' not found – skipped.")
    df = pd.DataFrame.from_records(records, columns=COLS)
    print(i)


    return df

ag_bh_merge = parse_merger_files("merger_files/BBHmergers_catalog.dat", "merger_files/BBHmergers_SSC.dat")

ag_bh_merge # no model paths

148 2.0 20.0 0.02 32 1054.50903668 787127.0 4453036.0 25.99104 28.4

0


,model_num,rv,rg,Z,N,merger_time,id1,id2,m1,m2,...,spin2,v_kick,v_esc,merger_channel,id3,m3,spin3,a_final,e_final,e_10hz
0,12.0,0.5,2.0,0.020,16.0,119.627165,1710471.0,2066202.0,44.6127,29.146100,...,0.0,111.792000,103.1810,4.0,1710473.0,70.4603,0.662207,0.000243,0.497284,-1.000000
1,12.0,0.5,2.0,0.020,16.0,145.914016,1680871.0,1702912.0,72.3486,64.708900,...,0.0,31.916700,101.7360,2.0,1680871.0,130.4770,0.684759,0.006803,0.142925,-1.000000
2,12.0,0.5,2.0,0.020,16.0,165.892453,1710980.0,1724282.0,22.5907,16.455800,...,0.0,86.674700,101.8880,4.0,1710982.0,37.2402,0.672892,0.000115,0.682157,-1.000000
3,12.0,0.5,2.0,0.020,16.0,286.406857,1680874.0,1681265.0,130.4770,102.868000,...,0.0,596.850000,97.0968,2.0,1680874.0,224.1360,0.590772,0.006825,0.003570,-1.000000
4,12.0,0.5,2.0,0.020,16.0,294.537586,1757186.0,1861303.0,12.7095,7.646160,...,0.0,128.374000,95.5495,2.0,1757186.0,19.4740,0.652218,1.194780,0.999843,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11065,149.0,2.0,20.0,0.002,1.0,3994.854440,1012119.0,9149421.0,-1.0000,29.966549,...,-1.0,0.000000,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.017292
11066,149.0,2.0,20.0,0.002,1.0,4038.046583,1687518.0,8243294.0,-1.0000,46.622272,...,-1.0,0.683903,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.091357
11067,149.0,2.0,20.0,0.002,1.0,4106.425254,2107286.0,10845792.0,-1.0000,16.656953,...,-1.0,0.000000,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.041849
11068,149.0,2.0,20.0,0.002,1.0,4139.542502,7659600.0,10623895.0,-1.0000,29.087456,...,-1.0,0.000000,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.090165


## tagging bbh mergers with model path

In [6]:
def process_dataframe_by_N(df):

    
    # Step 1: Sort dataframe by 'N'
    df_sorted = df.sort_values('N').copy()
    
    # Step 2: Split into two frames
    df_less_than_10 = df_sorted[df_sorted['N'] < 10].copy()
    df_greater_equal_10 = df_sorted[df_sorted['N'] >= 10].copy()
    
    print(f"Rows with N < 10: {len(df_less_than_10)}")
    print(f"Rows with N >= 10: {len(df_greater_equal_10)}")
    
    # Step 3: For N >= 10 table, change N to integer
    if not df_greater_equal_10.empty:
        df_greater_equal_10['N'] = df_greater_equal_10['N'].astype(int)
        print("Converted N >= 10 values to integers")
    
    # Step 4: Apply model_path creation to both dataframes
    # For N < 10 frame
    if not df_less_than_10.empty:
        df_less_than_10['model_path'] = ('data\\N' + 
                                        df_less_than_10['N'].astype(str) + 
                                        'rv' + df_less_than_10['rv'].astype(str) + 
                                        'rg' + df_less_than_10['rg'].astype(str) + 
                                        'Z' + df_less_than_10['Z'].astype(str))
    
    # For N >= 10 frame
    if not df_greater_equal_10.empty:
        df_greater_equal_10['model_path'] = ('data\\N' + 
                                            df_greater_equal_10['N'].astype(str) + 
                                            'rv' + df_greater_equal_10['rv'].astype(str) + 
                                            'rg' + df_greater_equal_10['rg'].astype(str) + 
                                            'Z' + df_greater_equal_10['Z'].astype(str))
    
    # Step 5: Convert N values in second dataframe back to floats
    if not df_greater_equal_10.empty:
        df_greater_equal_10['N'] = df_greater_equal_10['N'].astype(float)
        print("Converted N >= 10 values back to floats")
    
    # Step 6: Combine the two frames again
    df_combined = pd.concat([df_less_than_10, df_greater_equal_10], ignore_index=True)
    
    # Sort by original order if needed, or keep sorted by N
    df_final = df_combined.sort_values('N').reset_index(drop=True)
    
    print(f"Final combined dataframe shape: {df_final.shape}")
    
    return df_final



In [24]:
ag_bh_merge = process_dataframe_by_N(ag_bh_merge)


Rows with N < 10: 3863
Rows with N >= 10: 7207
Converted N >= 10 values to integers
Converted N >= 10 values back to floats
Final combined dataframe shape: (11070, 22)


In [28]:
y = np.array(ag_bh_merge['model_num'].unique())
y

array([149.,  61.,  49.,  73., 141., 121.,  77.,  65.,  69.,  81.,  85.,
        57.,  37.,  89.,  45.,  53.,  93.,  97.,  41., 137., 101., 117.,
       105., 133., 109., 129., 113., 125.,  62.,  78.,  86.,  58.,  66.,
        74.,  38.,  70.,  42.,  90., 142.,  54.,  46.,  94., 138., 102.,
        98., 106., 134., 110., 130., 114., 118.,  50., 126., 122.,  82.,
       119.,  35., 123., 127.,  51., 115., 111., 131., 107., 103., 135.,
       139.,  99.,  47.,  55.,  95.,  91., 143.,  87.,  43.,  59.,  83.,
        79.,  75.,  39.,  63.,  71.,  67.,  12.,  64.,  68.,  72.,  60.,
        76.,  80.,  84.,  56.,  88.,  92.,  96.,  52., 100., 104., 108.,
       112., 116., 120.,  48., 124., 128., 132., 136., 140.,  44., 144.,
        40.,  36., 146., 147., 145., 148.])

# Unit Coversion factor extraction

In [8]:
def load_conversion_factors(model_path):

    
    conv_file_path = os.path.join(model_path, 'initial.conv.sh')

    # Load conversion factors using cmc_parser - exactly as you showed
    conv_file = cp.conversion_file(conv_file_path)
        
    # Extract the conversion factors you demonstrated
    conversion_factors = {
        'mass_msun_mstar': conv_file.mass_msun_mstar,  # For converting masses
        'time_myr': conv_file.time_myr,                # For converting times 
        'length_parsec': conv_file.length_parsec,      # For converting radii
        'conv_obj': conv_file                          # Store full object for reference
    }
        
    return conversion_factors

# discover models

In [9]:

def parse_model_name(model_name: str):
    """
    Extract N, rv, rg and Z from directory names such as
    'N0.8rv0.75rg20Z0.002'  or  'N8e5rv0.75rg20z0.002_extra'.
    """
    # digits, dots, +, -, or scientific-notation letters
    num = r'([0-9.eE+-]+)'

    # ^ anchors at start; we don’t anchor at end so trailing text is allowed
    pattern = rf'^N{num}rv{num}rg{num}Z{num}'

    m = re.match(pattern, model_name, flags=re.IGNORECASE)
    if not m:
        print(f"Warning: Could not parse {model_name}")
        return None

    return {
        'N':  float(m.group(1)),
        'rv': float(m.group(2)),
        'rg': float(m.group(3)),
        'Z':  float(m.group(4)),
        'model_name': model_name
    }

In [10]:
def discover_models(data_directory):
    # Need to implement filtering
    model_dirs=[]
    for item in os.listdir(data_directory):
        item_path = os.path.join(data_directory, item)
        if os.path.isdir(item_path):
            parsed = parse_model_name(item)
            if parsed:
                parsed['path'] = item_path
                model_dirs.append(parsed)
        models_df = pd.DataFrame(model_dirs)
    return models_df

model_list = discover_models('data')
model_list

,N,rv,rg,Z,model_name,path
0,16.0,0.5,2.0,0.0020,N16rv0.5rg2.0Z0.002,data\N16rv0.5rg2.0Z0.002
1,16.0,0.5,2.0,0.0200,N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02
2,16.0,0.5,20.0,0.0020,N16rv0.5rg20.0Z0.002,data\N16rv0.5rg20.0Z0.002
3,16.0,0.5,20.0,0.0200,N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02
4,16.0,0.5,8.0,0.0020,N16rv0.5rg8.0Z0.002,data\N16rv0.5rg8.0Z0.002
...,...,...,...,...,...,...
140,8.0,4.0,20.0,0.0020,N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002
141,8.0,4.0,20.0,0.0200,N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02
142,8.0,4.0,8.0,0.0002,N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002
143,8.0,4.0,8.0,0.0020,N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002


# Full individual model processing

In [11]:
def full_model(model_path,ag_bh_merge,latest=True):
    #getting the original frames
    all_bh=parse_bhformation_dat(model_path)
    all_col= parse_collision_log(model_path)
    all_merge=parse_merger_file(model_path)

    # getting the merger frame
    all_bbh_merge = ag_bh_merge[ag_bh_merge['model_path']==model_path]
    # unit conversion
    conv_factors=load_conversion_factors(model_path)
    myr = conv_factors['time_myr']
    parsec = conv_factors['length_parsec']
    msun = conv_factors['mass_msun_mstar']

    #colision units
    all_col['time']=all_col['time']*myr# convert to MYr
    all_col['r']=all_col['r']*parsec # convert to Parsec
    
    #all bh units
    all_bh['time']=all_bh['time']*myr# convert to MYr
    all_bh['r']=all_bh['r']*parsec # convert to Parsec
    
    #all merge units
    all_merge['time']=all_merge['time']*myr# convert to MYr
    all_merge['r']=all_merge['r']*parsec # convert to Parsec

    #dropping all 0 ids
    all_bh = all_bh[all_bh['ID'] != 0]
    all_col = all_col[all_col['idm'] != 0]
    all_merge = all_merge[all_merge['id_rem'] != 0]

    #collision mass features 
    all_col['smaller_mass'] = all_col[['m1', 'm2']].min(axis=1)
    all_col['larger_mass'] = all_col[['m1', 'm2']].max(axis=1)
    all_col['mass_ratio'] = all_col['smaller_mass'] / all_col['larger_mass']
    # Collision and Merge BHs 
    col_bh = pd.merge(all_col,all_bh,left_on='idm',right_on='ID',how='inner')
    merge_bh = pd.merge(all_merge,all_bh,left_on='id_rem',right_on='ID',how='inner')

    # model path tagging
    all_bh['model'] = model_path
    all_col['model'] = model_path
    all_merge['model'] = model_path
    col_bh['model'] = model_path
    merge_bh['model'] = model_path
    
    #latest event merger fix

    if latest == True:
        filtered_df = merge_bh[merge_bh['time_y'] >= merge_bh['time_x']]

        merge_bh = filtered_df.loc[filtered_df.groupby('ID')['time_x'].idxmax()]


    #latest event col fix 9im iffy about this
    if latest == True:
        result_rows = []
        for id_val, id_group in col_bh.groupby('ID'):
        
            # Step 2: For each ID, group by time_y
            for time_y_val, time_y_group in id_group.groupby('time_y'):
            
                # Step 3: Drop rows where time_x > time_y
                valid_rows = time_y_group[time_y_group['time_x'] <= time_y_val]
            
                # Step 4: If valid rows exist, take the one with latest time_x
                if not valid_rows.empty:
                    latest_row = valid_rows.loc[valid_rows['time_x'].idxmax()]
                    result_rows.append(latest_row)
    col_bh = pd.DataFrame(result_rows)

 
    
    #output

    all_frames = {'all_bh':all_bh,'col':all_col,'merge':all_merge,'col_bh':col_bh,'merge_bh':merge_bh,'all_bbh_merge':all_bbh_merge}
    return all_frames

# testing

In [12]:
model_list

,N,rv,rg,Z,model_name,path
0,16.0,0.5,2.0,0.0020,N16rv0.5rg2.0Z0.002,data\N16rv0.5rg2.0Z0.002
1,16.0,0.5,2.0,0.0200,N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02
2,16.0,0.5,20.0,0.0020,N16rv0.5rg20.0Z0.002,data\N16rv0.5rg20.0Z0.002
3,16.0,0.5,20.0,0.0200,N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02
4,16.0,0.5,8.0,0.0020,N16rv0.5rg8.0Z0.002,data\N16rv0.5rg8.0Z0.002
...,...,...,...,...,...,...
140,8.0,4.0,20.0,0.0020,N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002
141,8.0,4.0,20.0,0.0200,N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02
142,8.0,4.0,8.0,0.0002,N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002
143,8.0,4.0,8.0,0.0020,N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002


In [13]:
x = pd.DataFrame()

x['path']= ag_bh_merge['model_path'].unique()
x

,path
0,data\N1.0rv2.0rg20.0Z0.002
1,data\N2.0rv1.0rg20.0Z0.002
2,data\N2.0rv1.0rg8.0Z0.0002
3,data\N2.0rv2.0rg2.0Z0.02
4,data\N2.0rv2.0rg8.0Z0.0002
...,...
111,data\N16rv0.5rg20.0Z0.02
112,data\N32rv1.0rg20.0Z0.02
113,data\N32rv2.0rg20.0Z0.0002
114,data\N32rv1.0rg20.0Z0.0002


In [14]:
pd.merge(left=model_list,right=x,left_on=model_list['path'],right_on=x['path'].unique(),how='inner' )

,key_0,N,rv,rg,Z,model_name,path_x,path_y
0,data\N16rv0.5rg2.0Z0.02,16.0,0.5,2.0,0.0200,N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02
1,data\N16rv0.5rg20.0Z0.02,16.0,0.5,20.0,0.0200,N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02
2,data\N16rv1.0rg2.0Z0.0002,16.0,1.0,2.0,0.0002,N16rv1.0rg2.0Z0.0002,data\N16rv1.0rg2.0Z0.0002,data\N16rv1.0rg2.0Z0.0002
3,data\N16rv1.0rg2.0Z0.002,16.0,1.0,2.0,0.0020,N16rv1.0rg2.0Z0.002,data\N16rv1.0rg2.0Z0.002,data\N16rv1.0rg2.0Z0.002
4,data\N16rv1.0rg2.0Z0.02,16.0,1.0,2.0,0.0200,N16rv1.0rg2.0Z0.02,data\N16rv1.0rg2.0Z0.02,data\N16rv1.0rg2.0Z0.02
...,...,...,...,...,...,...,...,...
110,data\N8.0rv4.0rg20.0Z0.002,8.0,4.0,20.0,0.0020,N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002
111,data\N8.0rv4.0rg20.0Z0.02,8.0,4.0,20.0,0.0200,N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02
112,data\N8.0rv4.0rg8.0Z0.0002,8.0,4.0,8.0,0.0002,N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002
113,data\N8.0rv4.0rg8.0Z0.002,8.0,4.0,8.0,0.0020,N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002


In [15]:
# Elements in x['path'] but NOT in model_list['path']
x_not_in_model = x[~x['path'].isin(model_list['path'])]

# Elements in model_list['path'] but NOT in x['path']  
model_not_in_x = model_list[~model_list['path'].isin(x['path'])]

# Just the path values as lists
x_only_paths = x_not_in_model['path'].tolist()
model_only_paths = model_not_in_x['path'].tolist()

# print("Paths in x but not in model_list:", x_only_paths)
# print("Paths in model_list but not in x:", model_only_paths)
x_only_paths

['data\\N1.0rv2.0rg20.0Z0.002']

In [30]:
print(model_only_paths)

['data\\N16rv0.5rg2.0Z0.002', 'data\\N16rv0.5rg20.0Z0.002', 'data\\N16rv0.5rg8.0Z0.002', 'data\\N16rv0.5rg8.0Z0.02', 'data\\N2.0rv0.5rg2.0Z0.0002', 'data\\N2.0rv0.5rg2.0Z0.002', 'data\\N2.0rv0.5rg2.0Z0.02', 'data\\N2.0rv0.5rg20.0Z0.0002', 'data\\N2.0rv0.5rg20.0Z0.002', 'data\\N2.0rv0.5rg20.0Z0.02', 'data\\N2.0rv0.5rg8.0Z0.0002', 'data\\N2.0rv0.5rg8.0Z0.002', 'data\\N2.0rv0.5rg8.0Z0.02', 'data\\N4.0rv0.5rg2.0Z0.0002', 'data\\N4.0rv0.5rg2.0Z0.002', 'data\\N4.0rv0.5rg2.0Z0.02', 'data\\N4.0rv0.5rg20.0Z0.0002', 'data\\N4.0rv0.5rg20.0Z0.002', 'data\\N4.0rv0.5rg20.0Z0.02', 'data\\N4.0rv0.5rg8.0Z0.0002', 'data\\N4.0rv0.5rg8.0Z0.002', 'data\\N4.0rv0.5rg8.0Z0.02', 'data\\N8.0rv0.5rg2.0Z0.0002', 'data\\N8.0rv0.5rg2.0Z0.002', 'data\\N8.0rv0.5rg2.0Z0.02', 'data\\N8.0rv0.5rg20.0Z0.0002', 'data\\N8.0rv0.5rg20.0Z0.002', 'data\\N8.0rv0.5rg8.0Z0.0002', 'data\\N8.0rv0.5rg8.0Z0.002', 'data\\N8.0rv0.5rg8.0Z0.02']
